In [3]:
!pip install PySide

    Complete output from command python setup.py egg_info:
    only these python versions are supported: [(2, 6), (2, 7), (3, 2), (3, 3), (3, 4)]
    
    ----------------------------------------


Command "python setup.py egg_info" failed with error code 1 in C:\Users\CHRIST~1\AppData\Local\Temp\pip-build-q9psopxr\PySide\


In [1]:
from cairosvg import svg2png

svg2png(bytestring=svg_code,write_to='output.png')

In [75]:
def GetScore(title, filename):
    import statistics as st
    import numpy as np
    import pygal as pg
    import pptx

    import openpyxl as op
    columns = ['A','B','C','D','E','F']
    pestle = {}

    wb2 = op.load_workbook(r'C:\Users\Christopher\Documents\MAN6721\PESTLE-S.xlsx')
    #print(wb2.get_sheet_names())
    sheet_ranges = wb2['Sheet1']

    for i in columns:
        temp = {}
        for x in range(2,100):
            cell = i+str(x)
            try:
                #print(sheet_ranges[i+'1'].value.lower(),sheet_ranges[cell].value.lower())
                temp[sheet_ranges[cell].value.lower()]=0
            except:
                x=100

        pestle[sheet_ranges[i+'1'].value.lower()]=temp
    #print(pestle)



    totalkeywords=0
    
    wordcounts={}
    wordlengths={}
    ls = []
    fin=open(filename,'r')
    for f in fin.readlines():
        ls.append(f.split())
    for x in ls:
        for i in x:
            tempi=""
            i=i.lower().strip()
            for j in i:
                if j.isalpha():
                    tempi += j
            i=tempi
            for x in pestle:
                for y in pestle[x]:
                    if i == y or i == y+'s' or i == y+'es' or i==y[:-1]+'ies':
                        pestle[x][y] += 1
                        totalkeywords+=1
                        #print(pestle[x])
            try:
                wordcounts[i] += 1
                wordlengths[i] = len(i)
            except:
                wordcounts[i] = 1
    loop = 0
    Score={}


    for x in pestle:
        #print('---------------------'+x+'-----------------------')
        tempwords=[]
        tempscore = 0
        Score[x]={'Words':[],'Score':0}
        total = 0
        for z in reversed(sorted(pestle[x], key=pestle[x].get)):
            try:
                total += pestle[x][z]
            except:
                total+=0
        for y in reversed(sorted(pestle[x], key=pestle[x].get)):
            try:
                if loop>=5:
                    raise
            except:
                break
            else:
                word = y+":"
                tempscore += pestle[x][y]
                tempwords.append(y)
                #print(word.ljust(20),pestle[x][y])
                loop+=1
        Score[x]['Words']=tempwords
        Score[x]['Score']=tempscore/totalkeywords
        loop = 0
    scorelabels = []
    scorenumbers = []
    for x in pestle:
        scorelabels.append(x)
        scorenumbers.append(Score[x]['Score'])


    radar_chart = pg.Radar(range=(.01, max(scorenumbers)+.1))
    radar_chart.title = title+' - PESTLE Score'
    radar_chart.x_labels = scorelabels
    radar_chart.add('Score', scorenumbers)
    #radar_chart.add('Actual Spending', [70, 50, 40, 10, 17, 8, 10])
    #radar_chart.render_to_file('radar_chart.svg')
    radar_chart.render_to_png(title+'-radar_chart.png')
    #print(title)
    return Score





In [85]:
def createppt(title, filename):
    ScoreDiff=0
    IndustryScore = GetScore('Industry', r"C:\Users\Christopher\Documents\MAN6721\All.txt")
    CompanyScore = GetScore(title, filename)
    #SLD_LAYOUT_TITLE_AND_CONTENT = 1
    prs = pptx.Presentation('Template.pptx')
    prs.slides[0].shapes.title.text='PESTLE Company Analysis against Industry'
    prs.slides[0].shapes[1].text_frame.text=title

    #for shape in slide1.shapes:
    #    if shape.is_placeholder:
    #        phf = shape.placeholder_format
    #        print('%d, %s' % (phf.idx, phf.type))
    #placeholder = slide1.placeholders[10]  # idx key, not position
    #graphic_frame = placeholder.insert_table(rows=2, cols=2)


    for x in IndustryScore:
        slide1 = prs.slides.add_slide(prs.slide_layouts[4])
        for shape in slide1.shapes:
            #print(shape)
            shape.text = 'foobar'

        p1 = IndustryScore[x]['Words']
        p2 = CompanyScore[x]['Words']
        slide1.shapes[1].text='Industry - ' + str(IndustryScore[x]['Score'])
        slide1.shapes[3].text='Company - '+ str(CompanyScore[x]['Score'])
        ScoreDiff += abs(IndustryScore[x]['Score']-CompanyScore[x]['Score'])

        text_frame = slide1.shapes[2].text_frame
        text_frame.clear()  # remove any existing paragraphs, leaving one empty one

        p = text_frame.paragraphs[0]
        p.text = p1[0]

        text_frame2 = slide1.shapes[4].text_frame
        text_frame2.clear()  # remove any existing paragraphs, leaving one empty one

        pp = text_frame2.paragraphs[0]
        pp.text = p2[0]

        for para_str in p1[1:]:
            p = text_frame.add_paragraph()        
            p.text = para_str

        for para_str in p2[1:]:
            pp = text_frame2.add_paragraph()        
            pp.text = para_str

        title_placeholder = slide1.shapes.title
        title_placeholder.text = x.title()


    slide7 = prs.slides.add_slide(prs.slide_layouts[8])
    placeholder = slide7.placeholders[1]  # idx key, not position
    title_placeholder = slide7.shapes.title
    title_placeholder.text = title.title()+' - Radar Chart'
    text_frame3 = slide7.shapes[2].text_frame
    text_frame3.clear()
    p = text_frame3.paragraphs[0]
    p.text = 'Difference in Scores - '+str(ScoreDiff)
    picture = placeholder.insert_picture(title+'-radar_chart.png')
    prs.save(title+'-Analysis.pptx')

createppt('AmGen', r"C:\Users\Christopher\Documents\MAN6721\AmGen.txt")

C:\Program Files\Anaconda3\lib\site-packages\openpyxl\workbook\names\named_range.py:124: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")


In [12]:
#import cairosvg
#cairosvg.svg2png(file_obj=open(r"C:\Users\Christopher\Documents\MAN6915\MAN6915\radar_chart.svg", "rb"), write_to="radar.png")